<a href="https://colab.research.google.com/github/GerardoMunoz/ML/blob/main/Arrhythmia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resumen


1. Clasificadores vistos
 1. Respuesta más repetida
 1. Característica más acertada
 1. Vecino cercano
 1. Bayesiano
 1. Reg. Logística
 1. Redes neuronales (capas densas o convolucionales)

1. Entrenar y evaluar el modelo
 1. Dividir datos de entrenamiento y prueba
 1. Matriz de confusión 
 1. Verosimilitud (likelihood),  exactitud (accuracy) y error o pérdida (loss)

1. Compara los resultados obtenidos entre varios modelos 

 



##  Lectura de los datos
Para este ejemplo lee solamente la altura y el peso y crea 5 características con cada una. Un valor bajo 

In [1]:
#from google.colab import files 
#uploaded = files.upload()

Conjunto de datos

https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data

Descripción

https://archive.ics.uci.edu/ml/datasets/arrhythmia

Artículo

http://www.cs.bilkent.edu.tr/tech-reports/1998/BU-CEIS-9802.pdf

In [2]:
f_in = open("arrhythmia.data",'r')
arrhythmia = f_in.readlines()
f_in.close()
len(arrhythmia)

452

In [3]:
alturas_cm=[]
pesos_kg=[]
etiquetas=[]
for renglon in arrhythmia:
  datos_ren_str = renglon.split(',')
  try:
      altura_cm = int(datos_ren_str[2])
      if altura_cm > 200:
        continue
      peso_kg = int(datos_ren_str[3])
      etiqueta = int(datos_ren_str[-1])
  except:
     continue
  alturas_cm.append(altura_cm)
  pesos_kg.append(peso_kg)
  etiquetas.append(etiqueta)
#alturas_cm=np.array(alturas_cm)
#pesos_kg=np.array(pesos_kg)
#etiquetas=np.array(etiquetas)
alturas_cm[:10],min(alturas_cm),max(alturas_cm)

([190, 165, 172, 175, 190, 169, 160, 162, 168, 167], 105, 190)

In [4]:
pesos_kg[:10],min(pesos_kg),max(pesos_kg)

([80, 64, 95, 94, 80, 51, 52, 54, 56, 67], 10, 176)

In [5]:
etiquetas[:10],min(etiquetas),max(etiquetas)

([8, 6, 10, 1, 7, 14, 1, 1, 1, 10], 1, 16)

A continuación se agrupan algunos pesos y algunas alturas.

In [6]:
minima,maxima=min(pesos_kg),max(pesos_kg)
peso_car=[int((peso-minima)/10) for peso in pesos_kg  ]
num_peso_car=max(peso_car)
peso_car[:10],min(peso_car),max(peso_car)

([7, 5, 8, 8, 7, 4, 4, 4, 4, 5], 0, 16)

In [7]:
minima,maxima=min(alturas_cm),max(alturas_cm)
altura_car=[int((altura-minima)/10) for altura in alturas_cm  ]
num_altura_car=max(altura_car)
altura_car[:10],min(altura_car),max(altura_car)

([8, 6, 6, 7, 8, 6, 5, 5, 6, 6], 0, 8)

In [8]:
etiqueta_carac=[etiqueta-1 for etiqueta in etiquetas]
num_etiqueta_carac=max(etiqueta_carac)
etiqueta_carac[:10],min(etiqueta_carac),max(etiqueta_carac)

([7, 5, 9, 0, 6, 13, 0, 0, 0, 9], 0, 15)

In [9]:
import numpy as np
X=np.array([peso_car,altura_car]).T
X.shape

(450, 2)

In [10]:
y=np.array(etiqueta_carac)
y.shape

(450,)

## Separar datos de entrenamiento y de prueba

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((301, 2), (149, 2), (301,), (149,))

##  Etiqueta que más se repite

Este modelo sólo se usa como medida para comparar los otros modelos. 




In [12]:
cuenta_etiquetas={}
for etiqueta in y_train:
  if etiqueta in cuenta_etiquetas:
    cuenta_etiquetas[etiqueta] += 1
  else:
    cuenta_etiquetas[etiqueta] = 1


cuenta_etiquetas

{0: 171,
 14: 3,
 3: 13,
 1: 31,
 9: 28,
 5: 15,
 15: 14,
 2: 9,
 4: 7,
 8: 7,
 6: 2,
 13: 1}

Algunos modelos requieren balancear el conjunto de datos

In [13]:
suma=0
maximo=-1
eti_max=-1

for etiqueta in cuenta_etiquetas:
  valor=cuenta_etiquetas[etiqueta]
  suma += valor
  if valor > maximo:
    maximo = valor
    eti_max = etiqueta

f'Asumiendo siempre {eti_max} tiene una exctitud (accuracy) {maximo/suma}'

'Asumiendo siempre 0 tiene una exctitud (accuracy) 0.5681063122923588'

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, [0]*len(y_train))

0.5681063122923588

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, [0]*len(y_test))

0.4966442953020134

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, [0]*len(y_train))


array([[171,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 31,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 13,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 15,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 28,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 14,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, [0]*len(y_test))

array([[74,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

## Clasificador Bayesiano

Para ese ejemplo vamos a usar unn clasificador bayesiano categórico, https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html#sklearn.naive_bayes.CategoricalNB

Este clasificador requiere que si una característica tiene $n$ categorías, entonces estas estén codificadas entre $0$ y $n-1$   



In [18]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB()
clf.fit(X_train, y_train)
print(clf.predict(X[2:3]))

[0]


In [19]:
accuracy_score(y_train, clf.predict(X_train))

0.5813953488372093

In [20]:
accuracy_score(y_test, clf.predict(X_test))

0.4966442953020134

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, clf.predict(X_train))

array([[168,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [ 30,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 13,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  6,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0],
       [ 15,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  6,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [ 25,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 11,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   2]])

In [22]:
confusion_matrix(y_test, clf.predict(X_test))

array([[72,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1],
       [13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1],
       [10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [20,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

## Característica más acertada

Este modelo, por lo básico, usualmente sólo es usado para como referencia para comparar los otros modelos.


### Peso

In [23]:
peso_eti=np.zeros((num_peso_car+1,num_etiqueta_carac+1))
peso_eti.shape

(17, 16)

In [24]:
for i in range(len(peso_car)):
  #print(peso_car[i],etiqueta_carac[i])
  peso_eti[peso_car[i],etiqueta_carac[i]] +=1
peso_eti

array([[ 1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  3.,  0.,  0.,  0.,
         1.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  1.],
       [12.,  2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,
         0.,  0.,  1.],
       [54.,  4.,  0.,  1.,  1.,  4.,  0.,  0.,  1., 12.,  0.,  0.,  0.,
         2.,  0.,  2.],
       [63., 14.,  5.,  3.,  1., 10.,  1.,  0.,  1., 14.,  0.,  0.,  0.,
         0.,  2.,  4.],
       [56., 11.,  4.,  6.,  3.,  5.,  1.,  1.,  4., 11.,  0.,  0.,  0.,
         1.,  1.,  5.],
       [38.,  9.,  3.,  5.,  2.,  4.,  1.,  1.,  2.,  1.,  0.,  0.,  0.,
         0.,  1.,  5.],
       [14.,  2.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,
         0.,  1.,  3.],
       [ 5.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0

In [25]:
peso_eti.shape

(17, 16)

In [26]:
a_max_p=np.argmax(peso_eti,axis=1)
a_max_p

array([4, 9, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1])

In [27]:
X.shape

(450, 2)

In [28]:
def pred_car(X,i,respuestas):

  return [respuestas[int(dat)] for dat in X[:,i] ]

accuracy_score(y_train, pred_car(X_train,0,a_max_p))

0.5813953488372093

In [29]:
accuracy_score(y_test, pred_car(X_test,0,a_max_p))

0.5100671140939598

### Altura

In [30]:
alt_eti=np.zeros((num_altura_car+1,num_etiqueta_carac+1))
alt_eti.shape

(9, 16)

In [31]:
for i in range(len(altura_car)):
  #print(peso_car[i],etiqueta_carac[i])
  alt_eti[altura_car[i],etiqueta_carac[i]] +=1
alt_eti

array([[  0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,   0.,
          0.,   0.,   1.,   0.,   0.],
       [  1.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   2.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [ 13.,   3.,   0.,   0.,   0.,   2.,   0.,   0.,   1.,   2.,   0.,
          0.,   0.,   0.,   0.,   1.],
       [124.,  24.,   1.,   7.,   5.,   6.,   0.,   1.,   5.,  12.,   0.,
          0.,   0.,   0.,   3.,   5.],
       [ 72.,  15.,   9.,   4.,   2.,  15.,   1.,   0.,   1.,  25.,   0.,
          0.,   0.,   3.,   2.,  10.],
       [ 31.,   2.,   5.,   4.,   1.,   2.,   1.,   0.,   1.,   3.,   0.,
          0.,   0.,   0.,   0.,   3.],
       [  4.,   0.,   0.,   0.,   0.,   0.,   1.,   1.,   0.,   3.,   0.,
          0.,   0.,   

In [32]:
alt_eti.shape

(9, 16)

In [33]:
a_max_alt=np.argmax(alt_eti,axis=1)
a_max_alt

array([ 4,  9, 15,  9,  0,  0,  0,  0,  0])

In [34]:
X.shape

(450, 2)

In [35]:
def pred_car(X,i,respuestas):

  return [respuestas[int(dat)] for dat in X[:,i] ]

accuracy_score(y_train, pred_car(X_train,1,a_max_alt))

0.584717607973422

In [42]:
accuracy_score(y_test, pred_car(X_test,1,a_max_alt))

0.5100671140939598